In [ ]:
import sys

import gym
import safety_gym

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from garage.envs import GymEnv

sys.path.append('../robust_rewards_from_preferences')
import envs.custom_safety_envs
del sys.path[-1]

from replay_buffer import ReplayBuffer
from garage.trainer import Trainer
from dreamer import Dreamer
from utils import RandomPolicy
from garage.experiment.experiment import ExperimentContext
from garage.sampler import RaySampler, LocalSampler, DefaultWorker
import gym.envs.atari
from garage.sampler.worker_factory import WorkerFactory
import threading

import torch

from ruamel.yaml import YAML
from dotmap import DotMap

from garage import EpisodeBatch, TimeStepBatch
from garage.envs.wrappers import ClipReward, EpisodicLife,  FireReset, Grayscale,  MaxAndSkip, Noop,  Resize, StackFrames
from wrappers import Renderer

import os
from garage.experiment.deterministic import set_seed
import pickle
from models import WorldModel
from garage.torch import set_gpu_mode

import dowel
from dowel import logger, tabular
import torch.nn.functional as F

In [2]:
600+34*34

1756

In [3]:
mlp = MLP(input_shape=1756, out_shape=64, units=[400, 400, 400, 400], dist='gaussian', ind_dims=1, scale=None)

In [13]:
import numpy as np
import torch

In [ ]:
import gym
import minerl

In [ ]:
env = gym.make('MineRLTreechopVectorObf-v0')

In [ ]:
env.action_space

In [18]:
with torch.no_grad():
    mlp.net.out_layer.bias.data = torch.nn.Parameter(torch.ones(64))

In [24]:
mlp.net.out_layer.bias.dtype

torch.float32

In [2]:
from utils import update

In [3]:
from utils import set_config, get_config

In [4]:
set_config('defaults')
CONFIG = get_config()

In [5]:

# logger.add_output(
#     dowel.WandbOutput(
#         project='dreamer',
#         name='JUPYTER_RUN',
#         config=CONFIG,
#     )
# )


In [6]:


ctxt = ExperimentContext(snapshot_dir='./snapshot_dir', snapshot_mode='gap_overwrite', snapshot_gap=50)
snapshot_dir = ctxt.snapshot_dir

env = gym.envs.atari.AtariEnv(
    CONFIG.env.name, obs_type='image', frameskip=1,
    repeat_action_probability=0.25, full_action_space=False)
env = Noop(env, noop_max=30)
env = MaxAndSkip(env, skip=4)
# env = EpisodicLife(env)
if CONFIG.image.color_channels == 1:
    env = Grayscale(env)
env = Resize(env, CONFIG.image.height, CONFIG.image.height)
max_episode_length = 108000 / 4
env = Renderer(env, directory=os.path.join(snapshot_dir, 'videos'))
env = GymEnv(env, max_episode_length=max_episode_length, is_image=True)

set_seed(CONFIG.training.seed)

with open(os.path.join(snapshot_dir, 'env.pkl'), 'wb') as outfile:
    pickle.dump(env, outfile)

trainer = Trainer(ctxt)

buf = ReplayBuffer(env.spec)
policy = RandomPolicy(env.spec)
world_model = WorldModel(env.spec)

if CONFIG.training.sampler == "ray":
    Sampler = RaySampler
elif CONFIG.training.sampler == "local":
    Sampler = LocalSampler

sampler = Sampler(agents=policy,  # noqa: F841
                  envs=env,
                  max_episode_length=max_episode_length,
                  n_workers=4)

log_sampler = Sampler(agents=policy,  # noqa: F841
                  envs=env,
                  max_episode_length=max_episode_length,
                  n_workers=4)

#     log_sampler = Sampler(agents=policy,  # noqa: F841
#                           envs=env,
#                           max_episode_length=kwargs['max_episode_length'],
#                           n_workers=kwargs['n_workers'])

set_gpu_mode(False)

algo = Dreamer(
    env.spec,
    sampler=sampler,
    log_sampler=log_sampler,
    world_model=world_model,
    agent=policy,
    buf=buf,
)

trainer.setup(
    algo=algo,
    env=env,
)

# trainer.train(n_epochs=CONFIG.training.n_epochs)

/home/jamesborg/.pyenv/versions/3.8.7/envs/garage_2021.03/lib/python3.8/site-packages/garage/experiment/deterministic.py:36: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  warnings.warn(
/home/jamesborg/.pyenv/versions/3.8.7/envs/garage_2021.03/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [8]:
algo._initialize_dataset(trainer)
segs = algo.buffer.sample_segments(50)

In [29]:
import time
from garage.torch import global_device

In [48]:
def segs_to_batch(segs, env_spec):
    device = global_device()
    start = time.time()
    actions = torch.tensor(
        [env_spec.action_space.flatten_n(seg.actions) for seg in segs]
    ).type(torch.float).to(device)
    rewards = torch.tensor(
        [seg.rewards for seg in segs]).type(torch.float).to(device)
    
    discounts = (
        1 - torch.tensor([seg.terminals for seg in segs]).type(torch.float)
    ).to(device)
    print(time.time() -start)
    
    
    start = time.time()
    obs = torch.tensor(
        [seg.next_observations for seg in segs]).type(torch.float).to(device)
    print(time.time() -start)
    
    if CONFIG.image.color_channels == 1:
        obs = obs.unsqueeze(2)
    obs = obs / 255 - 0.5

    return obs, actions, rewards, discounts


In [76]:
start = time.time()
A = torch.tensor(np.random.rand(50, 50, 64, 64))
print(time.time() - start)

0.28697943687438965


In [78]:
A.element_size() * A.nelement() / 1000000000

0.0002048

In [70]:
list(A.shape)

[50, 64, 64]

In [72]:
A.reshape([10, 5] + list(A.shape)[1:]).shape

torch.Size([10, 5, 64, 64])

In [20]:
segs[0].observations.shape

(50, 64, 64)

In [74]:
200000 * 64 * 64

819200000

In [16]:
trainer.train(n_epochs=5)

/home/jamesborg/workspace/research/dreamer/dreamer.py:52: LoggerWarning: No outputs have been added to the logger.
  logger.log('INITIALIZING')
/home/jamesborg/workspace/research/dreamer/dreamer.py:52: LoggerWarning: Log data of type str was not accepted by any output
  logger.log('INITIALIZING')
  0%|          | 0/10 [00:00<?, ?it/s]/home/jamesborg/workspace/research/dreamer/dreamer.py:74: LoggerWarning: Log data of type TabularInput was not accepted by any output
  logger.log(tabular)
/home/jamesborg/workspace/research/dreamer/dreamer.py:76: TabularInputWarning: TabularInput {Buffer/BufferSize: type(int)} was not accepted by any output
  tabular.clear()
/home/jamesborg/workspace/research/dreamer/dreamer.py:76: TabularInputWarning: TabularInput {world_modelkl_loss: type(Tensor)} was not accepted by any output
  tabular.clear()
/home/jamesborg/workspace/research/dreamer/dreamer.py:76: TabularInputWarning: TabularInput {world_modelreward_loss: type(Tensor)} was not accepted by any outpu

In [17]:
eps = algo._sampler.obtain_exact_episodes(n_eps_per_worker=1, agent_update=algo.agent)

In [18]:
eps.observations.shape

(866, 64, 64)

In [19]:
eps.next_observations.shape

(866, 64, 64)

In [20]:
eps.actions.shape

(866,)

In [42]:
torch.tensor(0.) * torch.log(torch.tensor(0.) / torch.tensor(0.))

tensor(nan)

In [24]:
obs = torch.tensor(eps.observations).type(torch.float).unsqueeze(1).unsqueeze(0)
actions = torch.tensor(env.spec.action_space.flatten_n(eps.actions)).type(torch.float).unsqueeze(0)

In [34]:
out['posteriors'][0]['dist'].probs

tensor([[[0.0181, 0.0091, 0.0142,  ..., 0.0335, 0.0637, 0.0077],
         [0.0507, 0.0030, 0.0222,  ..., 0.0159, 0.0109, 0.1066],
         [0.0108, 0.0219, 0.0412,  ..., 0.0142, 0.0173, 0.0691],
         ...,
         [0.0144, 0.0265, 0.0157,  ..., 0.0069, 0.0612, 0.0121],
         [0.0477, 0.0194, 0.0428,  ..., 0.0613, 0.0289, 0.0058],
         [0.0487, 0.0028, 0.0134,  ..., 0.0307, 0.0822, 0.0255]]],
       grad_fn=<SoftmaxBackward>)

In [37]:
out['posteriors'][50]['sample']

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]]], grad_fn=<AddBackward0>)

In [35]:
out['posteriors'][50]['dist'].probs

tensor([[[0.0178, 0.0095, 0.0144,  ..., 0.0343, 0.0614, 0.0078],
         [0.0526, 0.0030, 0.0212,  ..., 0.0164, 0.0112, 0.1078],
         [0.0114, 0.0222, 0.0398,  ..., 0.0144, 0.0183, 0.0682],
         ...,
         [0.0146, 0.0276, 0.0159,  ..., 0.0069, 0.0636, 0.0123],
         [0.0477, 0.0193, 0.0427,  ..., 0.0597, 0.0286, 0.0059],
         [0.0462, 0.0029, 0.0137,  ..., 0.0311, 0.0846, 0.0257]]],
       grad_fn=<SoftmaxBackward>)

In [ ]:
segs, steps, channels, height, width = observations.shape
flattened_observations = observations.reshape(
    segs*steps, channels, height, width)
embedded_observations = self.image_encoder(
    flattened_observations).reshape(segs, steps, -1)
out = self.rssm.observe(embedded_observations, actions)
out['reward_dist'] = self.reward_predictor(out['feats'])
out['discount_dist'] = self.discount_predictor(out['feats'])
flattened_feats = out['feats'].reshape(segs*steps, self.feat_size)
mean = self.image_decoder(flattened_feats).reshape(
    segs, steps, channels, height, width)
norm = distributions.Normal(loc=mean, scale=1)
image_recon_dist = distributions.Independent(norm, 3)
assert image_recon_dist.batch_shape == (segs, steps)
out['image_recon_dist'] = image_recon_dist


In [13]:
image_recon = world_model.reconstruct(obs, actions)

In [46]:
image_recon

tensor([[[[0.4819, 0.4742, 0.4841,  ..., 0.4814, 0.4772, 0.4791],
          [0.4779, 0.4799, 0.4760,  ..., 0.4756, 0.4774, 0.4743],
          [0.4827, 0.4746, 0.4826,  ..., 0.4808, 0.4761, 0.4799],
          ...,
          [0.4764, 0.4827, 0.4769,  ..., 0.4790, 0.4789, 0.4765],
          [0.4774, 0.4763, 0.4792,  ..., 0.4731, 0.4783, 0.4730],
          [0.4797, 0.4835, 0.4845,  ..., 0.4768, 0.4787, 0.4777]]],


        [[[0.4819, 0.4744, 0.4840,  ..., 0.4815, 0.4773, 0.4794],
          [0.4779, 0.4800, 0.4760,  ..., 0.4755, 0.4773, 0.4744],
          [0.4828, 0.4745, 0.4828,  ..., 0.4809, 0.4762, 0.4798],
          ...,
          [0.4763, 0.4832, 0.4774,  ..., 0.4795, 0.4791, 0.4764],
          [0.4775, 0.4761, 0.4793,  ..., 0.4730, 0.4782, 0.4730],
          [0.4798, 0.4836, 0.4846,  ..., 0.4766, 0.4786, 0.4776]]],


        [[[0.4818, 0.4742, 0.4840,  ..., 0.4812, 0.4770, 0.4794],
          [0.4778, 0.4798, 0.4764,  ..., 0.4753, 0.4772, 0.4743],
          [0.4826, 0.4748, 0.4829,  ..

In [15]:
eps.lengths

array([180, 155, 177, 199], dtype=int32)

In [35]:
import importlib
import utils
importlib.reload(utils)
from utils import log_reconstructions

In [38]:
log_reconstructions(eps, env.spec, world_model, 1, os.path.join(snapshot_dir, 'videos'))

In [15]:
from video import export_video

In [31]:
export_video(np.tile(eps.observations[:146, ..., np.newaxis], (1, 1, 1, 3)), './video1.mp4')

In [40]:
import wandb

In [44]:
fname = './snapshot_dir/videos/reconstructed_1_0.mp4'

In [45]:
wandb.log({os.path.basename(fname): wandb.Video(fname),}, step=2)

In [30]:
.shape

(146, 64, 64, 3)

In [29]:
eps.observations[:146].shape

(146, 64, 64)

In [32]:
obs.shape

torch.Size([1, 844, 1, 64, 64])